In [1]:
!pip install datasets optuna scikit-learn

In [2]:
import gdown

file_id = '1Y9QFfJRUIEbrvLtQRZgX6W4O3z1IzDhc'
gdown.download(f'https://drive.google.com/uc?id={file_id}', '/content/DL_train_test_data.xlsx', quiet=False)

Downloading...From: https://drive.google.com/uc?id=1Y9QFfJRUIEbrvLtQRZgX6W4O3z1IzDhcTo: /content/DL_train_test_data.xlsx100%|██████████| 25.9M/25.9M [00:01<00:00, 25.4MB/s]

'/content/DL_train_test_data.xlsx'

In [3]:
# Load data
from transformers import AutoTokenizer, RobertaForSequenceClassification
from torch.nn import CrossEntropyLoss
from sklearn.metrics import classification_report, accuracy_score, f1_score
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
import optuna
from tqdm.notebook import tqdm
import numpy as np
from torch.optim import AdamW


model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
df = pd.read_excel('DL_train_test_data.xlsx')

# Encode severity levels
df['severity_label'] = df['severity_code']
print(df['severity_label'].unique())  # Ensure labels range from 0 to 6

# Combine short and long descriptions into a single text field
df['text'] = df['short_description'].fillna('') + ' ' + df['long_description'].fillna('')

# Tokenize data
tokenized_data = tokenizer(
    df['text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: The secret `HF_TOKEN` does not exist in your Colab secrets.To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.You will be able to reuse this secret in all of your notebooks.Please note that authentication is recommended but still optional to access public models or datasets.  warnings.warn(

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[2 6 1 4 5 0]

In [4]:

class BugDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {k: torch.tensor(v) for k, v in encodings.items()}
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)


labels = df['severity_label'].values
dataset = BugDataset(tokenized_data, labels)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=7)
model.to(device)

<ipython-input-4-bb1ea8a46638>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).  self.encodings = {k: torch.tensor(v) for k, v in encodings.items()}

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

RobertaForSequenceClassification(  (roberta): RobertaModel(    (embeddings): RobertaEmbeddings(      (word_embeddings): Embedding(50265, 768, padding_idx=1)      (position_embeddings): Embedding(514, 768, padding_idx=1)      (token_type_embeddings): Embedding(1, 768)      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)      (dropout): Dropout(p=0.1, inplace=False)    )    (encoder): RobertaEncoder(      (layer): ModuleList(        (0-11): 12 x RobertaLayer(          (attention): RobertaAttention(            (self): RobertaSdpaSelfAttention(              (query): Linear(in_features=768, out_features=768, bias=True)              (key): Linear(in_features=768, out_features=768, bias=True)              (value): Linear(in_features=768, out_features=768, bias=True)              (dropout): Dropout(p=0.1, inplace=False)            )            (output): RobertaSelfOutput(              (dense): Linear(in_features=768, out_features=768, bias=True)              (LayerNorm): Lay

In [ ]:
'''
OG Code for run 1 and 2. Only epoch, batchsize and learning rate are changed

'''


from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import WeightedRandomSampler
from torch.optim import AdamW
from tqdm import tqdm


# Training parameters
epochs = 10
# batch_size = best_hyperparams['batch_size']
# learning_rate = best_hyperparams['learning_rate']
batch_size = 16
learning_rate = 1e-4

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Validation
    model.eval()
    val_loss = 0
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    avg_val_loss = val_loss / len(val_loader)
    accuracy = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')  # Use 'weighted' for imbalanced classes

    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")
    print(classification_report(true_labels, preds))

100%|██████████| 2568/2568 [14:49<00:00,  2.89it/s]

Epoch 1/10 | Train Loss: 0.6615602668705527

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

Validation Loss: 0.6660Validation Accuracy: 0.8331Validation F1 Score: 0.7573              precision    recall  f1-score   support           1       0.00      0.00      0.00       233           2       0.83      1.00      0.91      8556           4       0.00      0.00      0.00       716           5       0.00      0.00      0.00       559           6       0.00      0.00      0.00       206    accuracy                           0.83     10270   macro avg       0.17      0.20      0.18     10270weighted avg       0.69      0.83      0.76     10270

100%|██████████| 2568/2568 [14:51<00:00,  2.88it/s]

...

100%|██████████| 2568/2568 [14:50<00:00,  2.88it/s]

Epoch 4/10 | Train Loss: 0.6537684573852849

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))

Validation Loss: 0.6632Validation Accuracy: 0.8331Validation F1 Score: 0.7573              precision    recall  f1-score   support           1       0.00      0.00      0.00       233           2       0.83      1.00      0.91      8556           4       0.00      0.00      0.00       716           5       0.00      0.00      0.00       559           6       0.00      0.00      0.00       206    accuracy                           0.83     10270   macro avg       0.17      0.20      0.18     10270weighted avg       0.69      0.83      0.76     10270

 58%|█████▊    | 1486/2568 [08:35<06:12,  2.90it/s]

In [ ]:
# Save the model
torch.save(model.state_dict(), 'optimized_model.pth')
from google.colab import files
files.download('optimized_model.pth')